# db_import_sp

This script is imported by the slicing script. It serves to make it easier to deal with the geochemical data in the excel file by extracting the correct columns through names rather than its integer location. It also serves to convert coordinates to UTM and pixel location

In [1]:
"""
Created on Wed Aug 29 12:45:10 2018

@author: kongstad
Script for importing and manipulating excel database
"""

import utm
import pandas as pd

class measurement:

    def __init__(self,db_path):
        self._db_path = db_path
        self.coord_Pixel =[]
        self.coord_UTM =[]
        
    def setPath(self,db_path):
        self._db_path=db_path
        
    def loadFile(self):
        """Load Excel Sheet"""
        self.xldb=pd.ExcelFile(self._db_path)
        # Load a sheet into a DataFrame by name
        self.rmultiaqr = self.xldb.parse('R-Multi-AqR')
        # Using integer location as its easier and slicing out exactly the rows i want
        self.data_names = self.rmultiaqr.iloc[0,:].values
        self.data_table = self.rmultiaqr.iloc[3:,:] 
            
    def extractData(self,key):
        if key == 'Coordinates':
             # Since the dataset is not import friendly, this bit of code sets the 4th row
             # The one containing the categories, as the "key" kategori. These lines assign
             # The otherwise split coordinates (lat/lon) into a combined tuple under the designation
             # Coordinates.
             col = list(self.data_names).index(key)
             print('Extracted Column: ' + str(col))
             return [tuple(x) for x in self.data_table[['Unnamed: '+ str(col+1),'Unnamed: '+ str(col)]].values]
        else:
            # The remaning columns are assigned the name in the 4th row.
            col = list(self.data_names).index(key)
            print('Extracted Column: ' + str(col))
            return self.data_table['Unnamed: ' + str(col)].values.tolist()
            
        
    def UTM2Pixel(self,affine_transform):
        """Converts UTM coordinates to image pixel location"""
        for coord in self.coord_UTM:
            self.coord_Pixel.append(~affine_transform*coord)
            
          
    def LatLong2UTM(self):
        """Converts Latitude Longitutde coordinates to UTM"""
        for coord in self.coord_LatLon:
            lat,lon,zone_number,zone_letter = utm.from_latlon(coord[0],coord[1])
            self.coord_UTM.append((lat,lon))    